# **Cài đặt môi trường**

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (py

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
# Load model bloomz-1b1
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-1b1").to("cuda")
# Load tokenizer từ bloomz-1b1
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b1")

# **Tải dataset hỏi đáp web bán hàng**

In [ ]:
!wget https://raw.githubusercontent.com/phatjkk/data/main/LLM/Ecommerce_FAQ_Chatbot_dataset.json

--2023-06-24 10:02:22--  https://raw.githubusercontent.com/phatjkk/data/main/LLM/Ecommerce_FAQ_Chatbot_dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20978 (20K) [text/plain]
Saving to: ‘Ecommerce_FAQ_Chatbot_dataset.json.1’

Ecommerce_FAQ_Chatb 100%[===================>]  20.49K  --.-KB/s    in 0s      

2023-06-24 10:02:23 (82.8 MB/s) - ‘Ecommerce_FAQ_Chatbot_dataset.json.1’ saved [20978/20978]



In [ ]:
from datasets import load_dataset
from datasets import Dataset
data = load_dataset('json', data_files='Ecommerce_FAQ_Chatbot_dataset.json')
ds = Dataset.from_list(data["train"]["questions"][0])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Kết hợp câu hỏi và câu trả lời liền kề nhau để model học nội dung
# Mục đích là để khi đưa 1 đoạn câu hỏi, model sẽ tự nhiểu và gen ra câu trả lời
# phía sau
def merge_columns(example):
    example["prediction"] = example["question"] + " ->: " + str(example["answer"])
    return example

In [ ]:
# Map function trên vào dataset
ds = ds.map(merge_columns)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [ ]:
# Tokenizer/Véc tơ hóa văn bản (Chuyển văn bản thành số để training)
def tokeni(example):
    example["prediction"] = tokenizer(example["prediction"], return_tensors='pt', padding=True)['input_ids']
    return example
# Map function trên vào dataset
ds = ds.map(tokeni,batched=True)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [ ]:
# Hàm tính số lượng parameters có thể train được trên model
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# Tạo LoraConfig để áp dụng Lora vào việc training mô hình
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, #attention heads
    lora_alpha=16, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model_lora = get_peft_model(model, config)
print_trainable_parameters(model_lora)

trainable params: 1179648 || all params: 1066493952 || trainable%: 0.11060990995662018


In [ ]:
# Training model bằng hàm trainer
import transformers
trainer = transformers.Trainer(
    model=model_lora,
    train_dataset=ds["prediction"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    )
    ,data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model_lora.config.use_cache = True  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,3.713500
2,3.201300
3,3.317400
4,3.348000
5,2.609600
6,2.982500
7,3.449600
8,3.184900
9,3.298200
10,3.964700


TrainOutput(global_step=100, training_loss=2.298836829662323, metrics={'train_runtime': 12.1769, 'train_samples_per_second': 16.425, 'train_steps_per_second': 8.212, 'total_flos': 49361018167296.0, 'train_loss': 2.298836829662323, 'epoch': 2.5})

In [ ]:
tokenizer

BloomTokenizerFast(name_or_path='bigscience/bloomz-1b1', vocab_size=250680, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [ ]:
print(ds["question"][0])
print(ds["answer"][0])

How can I create an account?
To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.


In [ ]:
question = str(ds["question"][0]+" ->: ")
question

'How can I create an account? ->: '

In [ ]:
# In ra kết quả
inputs = tokenizer( question, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=1000)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

How can I create an account? ->:  If you do not have an account, please create one. Once you have created an account, you will be able to access your account and perform other actions.


In [ ]:
question2 = str(ds["question"][1]+" ->: ")
question2

'What payment methods do you accept? ->: '

In [ ]:
# In ra kết quả
inputs = tokenizer( question2, return_tensors="pt")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

What payment methods do you accept? ->:  You can accept payment methods such as credit cards, PayPal, and so on. Please note that payment methods are subject to restrictions and may not be available for certain payment methods.


In [ ]:
ask = "How to login?"
question3 = str(ask+" ->: ")
inputs = tokenizer( question3, return_tensors="pt")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

How to login? ->:  You can log in to your account by entering your username and password. If you have forgotten your password, you can reset it by clicking the "Reset Password" link. If you are logged into your account, you can also click the "Log In" link to log out.
